# Saving the maximun cluster as a separate NIFTI

In [2]:
import numpy as np
import nibabel as nib
import os
import pandas as pd

In [3]:
df = pd.read_csv('/data/bioprotean/SVD/colocalization/results_flip+rotated_jac.csv', delimiter=',', index_col=0)
df.head()

,cluster0,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,...,cluster40,cluster41,cluster42,cluster43,cluster44,cluster45,cluster46,cluster47,cluster48,cluster49
1,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.180536,0.819464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.074753,0.048895,0.876352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.045604,0.362953,0.552891,0.038552,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.045134,0.511989,0.273155,0.142924,0.026798,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Takes the general dataframe and the index and returns the value of cluster
def maxcluster_func(df, n_cluster):
    cluster_str = df.iloc[n_cluster-1,:].idxmax(axis=1)
    maxcluster_num = int(cluster_str.replace('cluster', ''))
    return maxcluster_num

In [36]:
# Takes the cluster number and the index and returns the masked cluster array
def maskcluster_func(maxcluster_num, index):
    img = nib.load('/data/bioprotean/SVD/clusters/nclusters_'+str(index)+'.nii')
    flat = np.array(img.dataobj).flatten()
    mask = np.where(flat==maxcluster_num, 1, 0)
    return mask

In [48]:
for i in range (1,51):
    maxcluster_num = maxcluster_func(df,i)
    mask = maskcluster_func(maxcluster_num,i)
    mask_rs = mask.reshape(67,58,41)
    img = nib.Nifti1Image(mask_rs.astype(np.int32), np.eye(4))
    img.header.get_xyzt_units()
    img.to_filename(os.path.join('build','/data/bioprotean/SVD/clusters/max/n'+str(i)+'_cluster'+str(maxcluster_num)+'.nii'))

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/SVD/clusters/max/
input='n11_cluster7.nii'
3dinfo $input -dxyz
3drefit -xyzscale 0.2 $input
3drefit -space ORIG $input
3drefit -orient ARS $input
@Align_Centers -base allen_200rs.nii -dset $input -no_cp